# Permeable Surface

This indicator was made for the 2023 blueprint.

Created by Amy Keister, last run by Amy Keister on 14 July 2023. It took 30 minutes to run. 

Because I am using a layer select, I have to run this in an arcPRO session with a map open.

Also, The NHD cathcments didn't cover all the islands, so I used some selection rules based on the size of islands and their overlap. These won't translate to other regions. So those steps will only work for the Virgin Islands and Puerto Rico area.

Also, these methods got way complex trying to get the islands to match the shoreline islands. If I have time I need to rethink all this. I was originaly doing the zonal statics at the original projection and cell size of the C-CAP. That seemed doable at first, but after working on adding in islands, this code has gotten really long and is taking a long time to run. It might not be worth it to do the analysis at the C-CAP cell size.

In [1]:
import os
import arcpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\CaribbeanPermeabl.gdb"

In [4]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\CaribbeanPermeableSurface.tif"

In [5]:
# define sub-indicator outputs to help with user support
PR= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfacePR.tif"
ST= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceST.tif"
SJ= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceSJ.tif"
SC= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceSC.tif"
PR1= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfacePR1.tif"
ST1= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceST1.tif"
SJ1= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceSJ1.tif"
SC1= r"D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\PermeableSurfaceSC1.tif"

In [6]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [7]:
# define inputs
catch= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\NHDPlus_H_National_Release_1_GDB.gdb\NHDPlusCatchment"

# The ccap data is served up seperately for PR and each USVI island.
# I did some pre-processing in another notebook named CCAPpre_JustExportTif.ipynb because
# ArcPro is not working with the original download rasters for some reason. It won't ID them 
# correctly and it also 
# won't perform processes on them correctly. I've tried many things, most of them work in ArcPro 
# outside of phython. Outside of python, if I reproject and bring the output into a geodatabase, 
# then ArcPro starts recognizing these rasters. But when I tried this in python it didn't work. 
# I tried mosaicing them first but it didn't make a raster that ArcPro can deal with. Finally I was 
# able to get copy raster to create rasters that ArcPro will recognize, but even with that, I had 
# to reboot my machine to get this copy to work correctly in python.
ccapPR= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copyPR.tif"
ccapST= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copyST.tif"
ccapSJ= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copySJ.tif"
ccapSC= r"F:\GIS_DATA\LanduseLandcover\CCAP\VIPR\copySC.tif"

# This islands layer was made using the CUSP data. There is another set of code that was used to create it.
isl= r"D:\SE_Blueprint_2023\Islands\Islands3Size.tif"

# there is a large missing catchment on the west coast of PR. I made this to fill it in
missing= r"F:\GIS_DATA\WaterResources\NHD\NHDPlus_H_National_Release_1_GDB\MissingCatchPR.shp"

# since the catchments don't cover the small islands, we need to make a layer for those islands
# I'm using the CUSP shoreline data, since it seems to have more recent shorelines
shore =r"F:\GIS_DATA\LanduseLandcover\CUSP_Shoreline\Southeast_Caribbean\Southeast_Caribbean.shp"

### Start Analysis

In [8]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [9]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\4_Indicators\CaribbeanPermeableSurface\CaribbeanPermeabl.gdb


### make islands for permeable surface

Since the catchments don't cover small islands, I'm going to make them from the shoreline data. This is tricky because the convert line to raster method is different than the convert polygon to raster method.

It is also tricky because, for this indicator, we don't want to duplicate islands already represented in the NHD plus, because then we create slivers aroudn those islands where the shoreline extent doesn't match the NHD extent.

In [10]:
# reclassify island layer
#out_raster = arcpy.sa.Reclassify(isl, "Value", "1 NODATA;2 1;3 1;4 1", "DATA"); out_raster.save("isla")

In [11]:
# convert islands to polygons
#arcpy.conversion.RasterToPolygon("isla", "islaP", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

In [12]:
# convert shore lines to polygons
# this doesn't work for the mainland of PR, because the shorelines do not connect at large river mouths
# I may come back and work on this later if we do want to try to improve the shorelines on the large islands
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.FeatureToPolygon(shore, "islandtest", None, "NO_ATTRIBUTES", None)

In [13]:
# add a field so we can dissolve
arcpy.management.CalculateField("islandtest", "dissolve", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\islandtest'>

In [14]:
# dissolve islands so that waterbodies on islands are included as a part of the island
arcpy.management.Dissolve("islandtest", "islandtestdis", "dissolve", None, "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\islandtestdis'>

In [15]:
# remove most island areas that overlap with or are near the NHD Plus catchments

# I'm doing this because I don't want to get into the hassle of trying to improve the NHD Plus Catchment shoreline

# first I do a select

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.SelectLayerByLocation("islandtestdis", "INTERSECT", catch, "10 Meters", "NEW_SELECTION", "INVERT")
    # add back in the south part of Culebra
    arcpy.management.SelectLayerByAttribute("islandtestdis", "ADD_TO_SELECTION", "Shape_Area > 2610000 And Shape_Area < 2620000", None)


In [16]:
# Now I make a new layer based on that select

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures("islandtestdis", "islandNoIntersect", '', None, None, None)

In [17]:
# Now I clear that selection
arcpy.management.SelectLayerByAttribute("islandtestdis", "CLEAR_SELECTION", "", None)

<Result 'islandtestdis_Layer8'>

### Make small islands shorelines match shoreline indicator 
now the "islandNoIntersect" polygon is all the polygons that are not captured in NHD that we do want to 
measure permeable surface on. Because convert polygon to raster is different to convert line to raster, I'm going to do both poygon and line conversions now, so at least my small island boundaries will match the small island boundareis of the shoreline indicator. This won't help shoreline matching for the big islands, but I just don't have an easy way to fix that for this indicator


In [18]:
# create a field to convert to raster
arcpy.management.CalculateField("islandNoIntersect", "raster", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\islandNoIntersect'>

In [19]:
# convert polygons to lines
#arcpy.management.PolygonToLine("islandNoIntersect", "ilandNoIntersectLine", "IDENTIFY_NEIGHBORS")
arcpy.management.PolygonToLine("islandNoIntersect", "islandNoIntersectLine", "IGNORE_NEIGHBORS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\islandNoIntersectLine'>

In [20]:
# convert polygons to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("islandNoIntersect", "raster", "islandNoIntersectR", 30)

In [21]:
# convert lines to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("islandNoIntersectLine", "raster", "islandNoIntersectLineR", 30)

In [22]:
# combine 
out_raster = arcpy.sa.CellStatistics("islandNoIntersectR;islandNoIntersectLineR", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("NotNHDislandsR")

In [23]:
# convert islands to polygons
arcpy.conversion.RasterToPolygon("NotNHDislandsR", "NotNHDislands", "NO_SIMPLIFY", "Value", "SINGLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\NotNHDislands'>

In [24]:
# add an id field
arcpy.management.CalculateField("NotNHDislands", "IslandID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\NotNHDislands'>

### Pull out impervious 

Since the CCAP data is finer resolution than the NLCD, and is binary, I running the analysis at the original scale

In [25]:
# pull out impervious surface to make binary rasters
# this didn't work because it made the background values natural

#out_raster = arcpy.sa.Con(ccapPR, 100, 0, "Value = 2"); out_raster.save("ccapPRimp")
#out_raster = arcpy.sa.Con(ccapST, 100, 0, "Value = 2"); out_raster.save("ccapSTimp")
#out_raster = arcpy.sa.Con(ccapSJ, 100, 0, "Value = 2"); out_raster.save("ccapSJimp")
#out_raster = arcpy.sa.Con(ccapSC, 100, 0, "Value = 2"); out_raster.save("ccapSCimp")

In [26]:
# pull out impervious surface to make binary rasters
out_raster = arcpy.sa.Con((arcpy.Raster(ccapPR) == 2), 100, arcpy.sa.Con((arcpy.Raster(ccapPR) > 2), 0, None)); out_raster.save("ccapPRimp1")
out_raster = arcpy.sa.Con((arcpy.Raster(ccapST) == 2), 100, arcpy.sa.Con((arcpy.Raster(ccapST) > 2), 0, None)); out_raster.save("ccapSTimp1")
out_raster = arcpy.sa.Con((arcpy.Raster(ccapSJ) == 2), 100, arcpy.sa.Con((arcpy.Raster(ccapSJ) > 2), 0, None)); out_raster.save("ccapSJimp1")
out_raster = arcpy.sa.Con((arcpy.Raster(ccapSC) == 2), 100, arcpy.sa.Con((arcpy.Raster(ccapSC) > 2), 0, None)); out_raster.save("ccapSCimp1")

### Zonal Statistics

I ended up getting overlap in the NHD and smaller non NHD islands anyway, because I wanted to make the small islands shorelines match the shoreline indicator. So I decided to just run the zonal statistics on the NHD seperatelly from the small, non NHD islands. 


In [27]:
# make a copy of the catchments just for VIPR
#with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
#    arcpy.management.CopyFeatures(catch, "catch", '', None, None, None)

In [28]:
# merge in the missing catchment
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Union([catch, missing], "catch", "ALL", None, "GAPS")

In [29]:
# create a field to use in zonal statistics
arcpy.management.CalculateField("catch", "UniqID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\catch'>

In [30]:
# calculate mean impervious surface per catchment
# running this in original projection/snap
with arcpy.EnvManager(outputCoordinateSystem=ccapPR, extent=ccapPR, snapRaster=ccapPR, cellSize=ccapPR):
    out_raster = arcpy.sa.ZonalStatistics("catch", "UniqID", "ccapPRimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanPr")
with arcpy.EnvManager(outputCoordinateSystem=ccapST, extent=ccapST, snapRaster=ccapST, cellSize=ccapST):
    out_raster = arcpy.sa.ZonalStatistics("catch", "UniqID", "ccapSTimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanST")
with arcpy.EnvManager(outputCoordinateSystem=ccapSJ, extent=ccapSJ, snapRaster=ccapSJ, cellSize=ccapSJ):
    out_raster = arcpy.sa.ZonalStatistics("catch", "UniqID", "ccapSJimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanSJ")
with arcpy.EnvManager(outputCoordinateSystem=ccapSC, extent=ccapSC, snapRaster=ccapSC, cellSize=ccapSC):
    out_raster = arcpy.sa.ZonalStatistics("catch", "UniqID", "ccapSCimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanSC")

In [31]:
# need to take 100 - the percent impervious value so it becomes percent permeable
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanPR"); out_raster.save("PR")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanST"); out_raster.save("ST")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanSJ"); out_raster.save("SJ")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanSC"); out_raster.save("SC")

In [32]:
# convert to integer
out_raster = arcpy.sa.Int("PR"); out_raster.save(PR)
out_raster = arcpy.sa.Int("ST"); out_raster.save(ST)
out_raster = arcpy.sa.Int("SJ"); out_raster.save(SJ)
out_raster = arcpy.sa.Int("SC"); out_raster.save(SC)

In [33]:
# reproject, convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample(PR, "PR30", "30 30", "MAJORITY")
    arcpy.management.Resample(ST, "ST30", "30 30", "MAJORITY")
    arcpy.management.Resample(SJ, "SJ30", "30 30", "MAJORITY")
    arcpy.management.Resample(SC, "SC30", "30 30", "MAJORITY")

In [34]:
# use cell statistics to combine instead of mosaic because of overlap between SJ and ST
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("PR30;ST30;SJ30;SC30", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("permeableNHD")

### Zonal Statistics

I ended up getting overlap in the NHD and smaller non NHD islands anyway, because I wanted to make the small islands shorelines match the shoreline indicator. So I decided to just run the zonal statistics on the NHD seperatelly from the small, non NHD islands. 


In [35]:
# calculate mean impervious surface per small island not covered by NHD
# running this in original projection/snap
with arcpy.EnvManager(outputCoordinateSystem=ccapPR, extent=ccapPR, snapRaster=ccapPR, cellSize=ccapPR):
    out_raster = arcpy.sa.ZonalStatistics("NotNHDislands", "IslandID", "ccapPRimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanPr1")
with arcpy.EnvManager(outputCoordinateSystem=ccapST, extent=ccapST, snapRaster=ccapST, cellSize=ccapST):
    out_raster = arcpy.sa.ZonalStatistics("NotNHDislands", "IslandID", "ccapSTimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanST1")
with arcpy.EnvManager(outputCoordinateSystem=ccapSJ, extent=ccapSJ, snapRaster=ccapSJ, cellSize=ccapSJ):
    out_raster = arcpy.sa.ZonalStatistics("NotNHDislands", "IslandID", "ccapSJimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanSJ1")
with arcpy.EnvManager(outputCoordinateSystem=ccapSC, extent=ccapSC, snapRaster=ccapSC, cellSize=ccapSC):
    out_raster = arcpy.sa.ZonalStatistics("NotNHDislands", "IslandID", "ccapSCimp1", "MEAN", "DATA", "CURRENT_SLICE", 90); out_raster.save("ZStatNHDCatchImpervMeanSC1")

In [36]:
# need to take 100 - the percent impervious value so it becomes percent permeable
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanPR1"); out_raster.save("PR1")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanST1"); out_raster.save("ST1")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanSJ1"); out_raster.save("SJ1")
out_raster = arcpy.sa.Minus(100, "ZStatNHDCatchImpervMeanSC1"); out_raster.save("SC1")

In [37]:
# convert to integer
out_raster = arcpy.sa.Int("PR1"); out_raster.save(PR1)
out_raster = arcpy.sa.Int("ST1"); out_raster.save(ST1)
out_raster = arcpy.sa.Int("SJ1"); out_raster.save(SJ1)
out_raster = arcpy.sa.Int("SC1"); out_raster.save(SC1)

In [38]:
# reproject, convert to 30 meters
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.Resample(PR1, "PR301", "30 30", "MAJORITY")
    arcpy.management.Resample(ST1, "ST301", "30 30", "MAJORITY")
    arcpy.management.Resample(SJ1, "SJ301", "30 30", "MAJORITY")
    arcpy.management.Resample(SC1, "SC301", "30 30", "MAJORITY")

In [39]:
# use cell statistics to combine instead of mosaic because of overlap between SJ and ST
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("PR301;ST301;SJ301;SC301", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("permeableIsl")

In [40]:
# use cell statistics to combine and choose minimum
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.CellStatistics("permeableNHD;permeableIsl", "MINIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("permeable")

In [41]:
# reclassify permeable surface to create binned version

# ArcGIS reclass includes the second value in the class for example,
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)

out_raster = arcpy.sa.Reclassify("permeable", "VALUE", "0 70 1;70 90 2;90 95 3;95 100 4", "DATA"); out_raster.save("indicator1")

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to extent

In [42]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 4:
        return '4 = >95% of catchment or small island permeable (likely high water quality and supporting most sensitive aquatic species)'
    elif value == 3:
        return '3 = >90-95% of catchment or small island permeable (likely declining water quality and supporting most aquatic species)'
    elif value == 2:
        return '2 = >70-90% of catchment or small island permeable (likely degraded water quality and not supporting many aquatic species)'
    elif value == 1:
        return '1 = ≤70% of catchment or small island permeable (likely degraded instream flow, water quality, and aquatic species communities)'   
"""

In [43]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("indicator1", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\indicator1'>

In [44]:
# set code block for next step
codeblock = """
def Reclass1(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 97
	if Value == 2:
		return 151
	if Value == 1:
		return 204
	else:
		return 255
		
def Reclass2(Value):
	if Value == 4:
		return 0
	if Value == 3:
		return 69
	if Value == 2:
		return 131
	if Value == 1:
		return 204
	else:
		return 255
		
def Reclass3(Value):
	if Value == 4:
		return 224
	if Value == 3:
		return 237
	if Value == 2:
		return 247
	if Value == 1:
		return 255
	else:
		return 255
"""

In [45]:
# calculate Red field
arcpy.management.CalculateField("indicator1", "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField("indicator1", "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField("indicator1", "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\SE_Blueprint_2023\\4_Indicators\\CaribbeanPermeableSurface\\CaribbeanPermeabl.gdb\\indicator1'>

In [46]:
# clip to extent
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.ExtractByMask("indicator1", Rextent); out_raster.save("indicator2")

In [47]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("indicator2", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [48]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

1084.78550863266
